# EXPLORATION DES DONNÉES DES POLLUANTS

**Objectifs :**

- Extraire la liste des polluants recherchés dans les prélèvements
- Sortir des premières analyses de ces données
- Disponibilité de la donnée
- Couverture du territoire : pour chaque polluant, quelle est la part des UDI avec analyse de chaque polluant
- Périodicité : pour chaque polluant, à quelle fréquence collecte-t-on
- Visualisation sur carte de la donnée pour chaque polluant. (obj : avoir une idée de la distribution, des valeurs min max etc)
- Analyse des polluants par seuils de dépassement

=> Analyser la donnée pour chaque polluant

In [11]:
# Nous commencons par importer les librairies nécessaires pour l'analyse des données et par connecter la base de données DuckDB

import duckdb

con = duckdb.connect(database="./../../database/data.duckdb", read_only=True)

In [17]:
# Liste des différentes tables présentes dans la base de données

tables = con.execute("SHOW TABLES").fetch_df()
tables

,name
0,edc_communes
1,edc_prelevements
2,edc_resultats


In [13]:
# Aperçu de la table "edc_communes"

edc_communes = con.execute("SELECT * FROM edc_communes LIMIT 5").fetch_df()
edc_communes

,inseecommune,nomcommune,quartier,cdreseau,nomreseau,debutalim,de_partition,de_ingestion_date
0,01001,ABERGEMENT-CLEMENCIAT (L'),-,001000556,BDS ST DIDIER/CHALARONNE,2010-09-07,2024,2025-02-05
1,01002,ABERGEMENT-DE-VAREY (L'),-,001000369,L'ABERGEMENT-DE-VAREY,2010-09-07,2024,2025-02-05
2,01004,AMBERIEU-EN-BUGEY,Vareilles,001000248,AMBERIEU VAREILLES,2010-09-07,2024,2025-02-05
3,01004,AMBERIEU-EN-BUGEY,St Germain_Brédevent,001000249,AMBERIEU SAINT GERMAIN DOUVRES,2010-09-07,2024,2025-02-05
4,01004,AMBERIEU-EN-BUGEY,ville,001000251,AMBERIEU VILLE,2010-09-07,2024,2025-02-05


In [14]:
# Aperçu de la table "edc_prelevements"

edc_prelevements = con.execute("SELECT * FROM edc_prelevements LIMIT 5").fetch_df()
edc_prelevements

,cddept,cdreseau,inseecommuneprinc,nomcommuneprinc,cdreseauamont,nomreseauamont,pourcentdebit,referenceprel,dateprel,heureprel,conclusionprel,ugelib,distrlib,moalib,plvconformitebacterio,plvconformitechimique,plvconformitereferencebact,plvconformitereferencechim,de_partition,de_ingestion_date
0,001,001000003,01007,AMBRONAY,None,None,None,00100139034,2024-01-23,11h27,Eau d'alimentation conforme aux limites de qua...,SYND. EAUX REGION D'AMBERIEU-EN-B,SERA - SYNDICAT DES EAUX DE LA REGION D'AMBERI...,SERA - SYNDICAT DES EAUX DE LA REGION D'AMBERI...,C,C,N,C,2024,2025-02-05
1,001,001000003,01007,AMBRONAY,None,None,None,00100139393,2024-02-13,12h49,Eau d'alimentation conforme aux exigences de q...,SYND. EAUX REGION D'AMBERIEU-EN-B,SERA - SYNDICAT DES EAUX DE LA REGION D'AMBERI...,SERA - SYNDICAT DES EAUX DE LA REGION D'AMBERI...,C,C,C,C,2024,2025-02-05
2,001,001000003,01007,AMBRONAY,001001304,TTP (CLG) AMBRONAY,100 %,00100139969,2024-03-13,10h46,Eau d'alimentation conforme aux exigences de q...,SYND. EAUX REGION D'AMBERIEU-EN-B,SERA - SYNDICAT DES EAUX DE LA REGION D'AMBERI...,SERA - SYNDICAT DES EAUX DE LA REGION D'AMBERI...,C,C,C,C,2024,2025-02-05
3,001,001000003,01007,AMBRONAY,None,None,None,00100140209,2024-04-05,07h27,Eau d'alimentation conforme aux exigences de q...,SYND. EAUX REGION D'AMBERIEU-EN-B,SERA - SYNDICAT DES EAUX DE LA REGION D'AMBERI...,SERA - SYNDICAT DES EAUX DE LA REGION D'AMBERI...,C,C,C,C,2024,2025-02-05
4,001,001000003,01007,AMBRONAY,None,None,None,00100140543,2024-05-13,12h43,Eau d'alimentation conforme aux exigences de q...,SYND. EAUX REGION D'AMBERIEU-EN-B,SERA - SYNDICAT DES EAUX DE LA REGION D'AMBERI...,SERA - SYNDICAT DES EAUX DE LA REGION D'AMBERI...,C,C,C,C,2024,2025-02-05


In [15]:
# Aperçu de la table "edc_resultats"

edc_resultats = con.execute("SELECT * FROM edc_resultats LIMIT 5").fetch_df()
edc_resultats

,cddept,referenceprel,cdparametresiseeaux,cdparametre,libmajparametre,libminparametre,libwebparametre,qualitparam,insituana,rqana,cdunitereferencesiseeaux,cdunitereference,limitequal,refqual,valtraduite,casparam,referenceanl,de_partition,de_ingestion_date
0,001,00100138503,ALTMICR,1370,ALUMINIUM TOTAL µG/L,Aluminium total µg/l,None,N,L,<10,µg/L,133,None,<=200 µg/L,0.00,7429-90-5,00100147233,2024,2025-02-05
1,001,00100138503,ASP,6489,ASPECT (QUALITATIF),Aspect (qualitatif),None,O,L,Aspect normal,SANS OBJET,X,None,None,0.00,None,00100147232,2024,2025-02-05
2,001,00100138503,ASP,6489,ASPECT (QUALITATIF),Aspect (qualitatif),None,O,L,Aspect normal,SANS OBJET,X,None,None,0.00,None,00100147233,2024,2025-02-05
3,001,00100138503,CL2LIB,1398,CHLORE LIBRE,Chlore libre,None,N,T,"0,30",mg(Cl2)/L,165,None,None,0.30,None,None,2024,2025-02-05
4,001,00100138503,CL2TOT,1399,CHLORE TOTAL,Chlore total,None,N,T,"0,34",mg(Cl2)/L,165,None,None,0.34,None,None,2024,2025-02-05


# 1 - EXPLORATION ET PREMIÈRES ANALYSES DE LA TABLE "edc_communes"

In [16]:
# Chargeons la table edc_communes dans un pandas dataframe, et calculons le nombre de communes

communes = con.table("edc_communes").to_df()
nombre_de_communes = communes.nunique()["inseecommune"]
print(f"nombre_de_communes = {nombre_de_communes}")

nombre_de_communes = 34809


In [34]:
# Chargeons la table edc_communes dans un pandas dataframe, et calculons le nombre de réseaux uniques

reseau = con.table("edc_communes").to_df()
nb_reseaux_uniques = reseau.nunique()["cdreseau"]
print(f"nombre_de_reseaux = {nb_reseaux_uniques}")

nombre_de_reseaux = 23184


In [ ]:
# Quelles sont les communes qui ont le plus de réseaux de prélèvements ?

communes_reseaux = (
    reseau.groupby("inseecommune")["cdreseau"].nunique().sort_values(ascending=False)
)
communes_reseaux

# Ajouter le nom des communes à la table précédente et grouper par inseecommune

communes_reseaux = communes_reseaux.reset_index()
communes_reseaux = communes_reseaux.merge(
    communes[["inseecommune", "nomcommune"]], on="inseecommune"
)
communes_reseaux = communes_reseaux.groupby("inseecommune").first().reset_index()
communes_reseaux.sort_values("cdreseau", ascending=False)

,inseecommune,cdreseau,nomcommune
17314,48027,19,MONT LOZERE ET GOULET
28793,73257,19,BELLEVILLE (LES)
17301,48009,18,PEYRE EN AUBRAC
28703,73150,17,PLAGNE TARENTAISE (LA)
34782,97416,16,SAINT-PIERRE
...,...,...,...
34806,97617,1,TSINGONI
34807,97701,1,SAINT-BARTHELEMY
14,01016,1,ARBIGNY
16,01019,1,ARMIX
